In [38]:
from impala.dbapi import connect
from impala.util import as_pandas
import pandas as pd
import pymysql
import xlwt
import re


In [39]:
def hive():
    cur = connect(host='192.168.0.252',port=10000,database='default').cursor()
    print('hive 连接成功')
    cur.execute("select * from dianpu_new where pub_date between '2019-04-28'and '2019-05-07'")
    hive_data = as_pandas(cur) 
    cur.close()
    
    return hive_data

In [40]:
hive_data=hive()

TTransportException: TTransportException(type=1, message="Could not connect to ('192.168.0.252', 10000)")

In [ ]:
hive_data[hive_data['dianpu_new.from_web']==2]

In [4]:
def dataframe_manage():
    df = hive_data
    df.columns = [(column.replace('dianpu_new.','')) for column in df.columns.tolist()]
    df = df.drop(df[df['address']==None].index.tolist())
    df.dropna(how='any',axis=0)
    return df

In [5]:
def address_menarge():
    df = dataframe_manage()
    address = df['address'].tolist()
    address_list = [str(i).split('-') for i in address]
    address1 = []
    address2 = []
    for i in  address_list:
        if i[0] == ''and i[1] != '':
                address1.append('1')
                address2.append(i[1])
        elif i[0] == 'None':
                address1.append('1')
                address2.append('1')
        elif i[1] == '' and i[0] !='':
                address1.append(i[0])
                address2.append('1')
        elif i[1] == '' and i[0] =='':
                address1.append('1')
                address2.append('1')

        else:
                address1.append(i[0])
                address2.append(i[1])
                

    df['address1'],df['address2'] = address1,address2
    df = df.drop(df[df['address1'].isin(['1','其他区'])].index.tolist())
    return df
    

In [6]:
def mysql():
    conn=pymysql.connect("192.168.0.252","data_user","first2018qaz","FBDdata",charset='utf8')
    sql = "SELECT * FROM house_info where date between '2019-04-22'and '2019-04-28'"
    sql_data = pd.read_sql(sql,conn)

    return  sql_data

In [20]:
#--------各网络平台发布数统计--------
#from_web 1为58，2为安居客
# wb 代表58 aj代表安居客 first代表菲斯特地产 ffx房发现
def Web_Analysis():
    df = address_menarge()
    df_wb = df.loc[df['from_web']==1]
    df_aj = df.loc[df['from_web']==2]
    df_first = df[df['agent_company'].isin(['菲斯特房地产','成都菲斯特房地产营销策划有限公司'])]
    df_ffx = mysql()
    
    wb_counts = df_wb.shape[0]
    aj_counts = df_aj.shape[0]
    first_counts = df_first.shape[0]
    ffx_counts = df_ffx.shape[0]
    counts_list = [wb_counts,aj_counts,first_counts,ffx_counts]
    
    
    wb_agent = df_wb['agent_name'].value_counts().shape[0]
    aj_agent = df_aj['agent_name'].value_counts().shape[0]
    first_agent = df_first['agent_name'].value_counts().shape[0]
    ffx_agent = df_ffx['agent_name'].value_counts().shape[0]
    agent_list = [wb_agent,aj_agent,first_agent,ffx_agent]
    
    avg = [round(wb_counts/wb_agent),round(aj_counts/aj_agent),round(first_counts/first_agent),round(ffx_counts/ffx_agent)]
    avg_sum = round(sum(avg))
    
    data = {'网站来源':['58同城','安居客','菲斯特','房发现平台','合计'],
            '发布条数':[wb_counts,aj_counts,first_counts,ffx_counts,sum(counts_list)],
            '经纪人数':[wb_agent,aj_agent,first_agent,ffx_agent,sum(agent_list)], 
            '人均发布数':[round(wb_counts/wb_agent),round(aj_counts/aj_agent),round(first_counts/first_agent),round(ffx_counts/ffx_agent),avg_sum]
           }
    df_excel = pd.DataFrame(data)
    return df_excel

In [21]:
Web_Analysis()

(1404, 0, 31, 48)

In [8]:
#地区发布占比
def Region_Analysis():
    df = address_menarge()
    region_names= df['address1'].value_counts().index.tolist()
    region_counts = df['address1'].value_counts().tolist()
    proportion = ['{:.2%}'.format(i/sum(region_counts)) for i in region_counts]
    rank = [i for i in range(1,len(region_names)+1)]
    data = {
        '排名':rank,
        '区域名称':region_names,
        '发布条数':region_counts,
        '占比':proportion
    }
    region_rank =pd.DataFrame(data)
    region_rank.cloumns = ['排名','区域名称','发布条数','占比']
    return region_rank

In [9]:
#总价处理
def Totalp_Analysis():
    df = address_menarge()
    rank = [i for i in range(1,9)]
    bin = [0,50,100,200,300,400,500,1000, float("inf")]#区间划分
    price_counts =  pd.cut(df['total_price'],bin).value_counts().tolist()
    price_cut = pd.cut(df['total_price'],bin).value_counts().index.tolist()
    price_names = ['总价: '+ str(i).replace(', ','-').strip('(').strip(']').replace('.0','').replace('0-50','<=50').replace('1000-inf','>1000') for i in price_cut]
    proportion = ['{:.2%}'.format(i/sum(price_counts)) for i in price_counts]
    data = {
        '排名':rank,
        '总价区间 单位:万':price_names,
        '发布条数':price_counts,
        '占比':proportion
    }
    totalp_rank = pd.DataFrame(data)
    totalp_rank.cloumns = ['排名','总价区间 单位:万','发布条数','占比']

    return totalp_rank

In [10]:
#单价处理
def Price_Analysis():
    df = address_menarge()
    rank = [i for i in range(1,9)]
    bin = [0,15000,20000,25000,30000,35000,40000,50000, float("inf")]#区间划分
    price_counts =  pd.cut(df['price'],bin).value_counts().tolist()
    price_cut = pd.cut(df['price'],bin).value_counts().index.tolist()
    price_names = ['单价: '+ str(i).replace('(0.0, 15000.0]','<1.5万').replace('(15000.0, 20000.0]','1.5-2万').replace('(20000.0, 25000.0]','2-2.5万').replace('(25000.0, 30000.0]','2.5-3万').replace('(30000.0, 35000.0]','3-3.5万').replace('(35000.0, 40000.0]','3.5-4万').replace('(40000.0, 50000.0]','4-5万').replace('(50000.0, inf','>5万') for i in price_cut]
    proportion = ['{:.2%}'.format(i/sum(price_counts)) for i in price_counts]
    data = {
        '排名':rank,
        '单价区间':price_names,
        '发布条数':price_counts,
        '占比':proportion
    }
    price_rank = pd.DataFrame(data)
    price_rank.cloumns = ['排名','单价区间','发布条数','占比']

    return price_rank

In [11]:
def Acreage_Analysis():
    df = address_menarge()
    rank = [i for i in range(1,6)]
    bin = [0,50,100,150,200, float("inf")]#区间划分
    acreage_counts =  pd.cut(df['acreage'],bin).value_counts().tolist()
    acreage_cut = pd.cut(df['acreage'],bin).value_counts().index.tolist()
    acreage_names = ['面积: '+ str(i).replace(', ','-').strip('(').strip(']').replace('.0','').replace('0-50','50').replace('200-inf','>200') for i in acreage_cut]
    proportion = ['{:.2%}'.format(i/sum(acreage_counts)) for i in acreage_counts]
    data = {
        '排名':rank,
        '面积区间':acreage_names,
        '发布条数':acreage_counts,
        '占比':proportion
    }
    acreage_rank = pd.DataFrame(data)
    acreage_rank.cloumns = ['排名','面积区间','发布条数','占比']
    return acreage_rank

In [12]:
#前5地区总价排名
def Top5totalp_Analysis():
    df = address_menarge()
    bin = [0,50,100,200,300,400,500,1000, float("inf")]#区间划分
    top5_region = df['address1'].value_counts().index.tolist()[:5]
    counts_list = []
    for k,v in enumerate(top5_region):
        Top = pd.cut(df[df['address1'] == top5_region[k]]['total_price'],bin).value_counts()
        df1 = pd.DataFrame(data={'bin':Top.index,'count':Top.values})
        df1.sort_values('bin',inplace=True)
        counts_list.append(df1['count'].tolist())
    
    
    data = {'总价区间    单位:万':['总价1：<=50','总价2：50-100','总价3：100-200','总价4：200-300','总价5：300-400','总价6：400-500','总价7：500-1000','总价8：>1000'],
            top5_region[0]:counts_list[0],
            top5_region[1]:counts_list[1],
            top5_region[2]:counts_list[2],
            top5_region[3]:counts_list[3],
            top5_region[4]:counts_list[4]
            }
    df = pd.DataFrame(data)
    return df

In [13]:
#前5地区单价排名
def Top5price_Analysis():
    df = address_menarge()
    bin = [0,15000,20000,25000,30000,35000,40000,50000,float("inf")]#区间划分
    top5_region = df['address1'].value_counts().index.tolist()[:5]
    counts_list = []
    for k,v in enumerate(top5_region):
        Top = pd.cut(df[df['address1'] == top5_region[k]]['price'],bin).value_counts()
        df1 = pd.DataFrame(data={'bin':Top.index,'count':Top.values})
        df1.sort_values('bin',inplace=True)
        counts_list.append(df1['count'].tolist())
    
    
    data = {'单价区间':['单价1：<1.5万','单价2：1.5-2万','单价3：2-2.5万','单价4：2.5-3万','单价5：3-3.5万','单价6：3.5-4万','单价7：4-5万','单价8：>5万'],
            top5_region[0]:counts_list[0],
            top5_region[1]:counts_list[1],
            top5_region[2]:counts_list[2],
            top5_region[3]:counts_list[3],
            top5_region[4]:counts_list[4]
            }
    df = pd.DataFrame(data)
    return df

In [14]:
#前5地区面积排名
def Top5acrege_Analysis():
    df = address_menarge()
    bin = [0,50,100,150,200,float("inf")]#区间划分
    top5_region = df['address1'].value_counts().index.tolist()[:5]
    counts_list = []
    for k,v in enumerate(top5_region):
        Top = pd.cut(df[df['address1'] == top5_region[k]]['acreage'],bin).value_counts()
        df1 = pd.DataFrame(data={'bin':Top.index,'count':Top.values})
        df1.sort_values('bin',inplace=True)
        counts_list.append(df1['count'].tolist())
    
    
    data = {'面积区间    单位：m²':['面积1：<50','面积2：50-100','面积3：100-150','面积4：150-200','面积5：>200'],
            top5_region[0]:counts_list[0],
            top5_region[1]:counts_list[1],
            top5_region[2]:counts_list[2],
            top5_region[3]:counts_list[3],
            top5_region[4]:counts_list[4]
            }
    df = pd.DataFrame(data)
    return df

In [15]:
with pd.ExcelWriter( r'D:/Excel/fang_tan.xlsx') as writer:
    Web_Analysis().to_excel(writer, sheet_name='网络来源',index=False)
    Region_Analysis().to_excel(writer, sheet_name='区域排名',index=False)
    Totalp_Analysis().to_excel(writer, sheet_name='总价排名',index=False)
    Price_Analysis().to_excel(writer, sheet_name='单价排名',index=False)
    Acreage_Analysis().to_excel(writer, sheet_name='面积排名',index=False)
    Top5totalp_Analysis().to_excel(writer, sheet_name='总价区域',index=False)
    Top5price_Analysis().to_excel(writer, sheet_name='单价区域',index=False)
    Top5acrege_Analysis().to_excel(writer, sheet_name='面积区域',index=False)
print('数据统计成功')

IndexError: At least one sheet must be visible